# Generating Formulas DataStream

**TODO** Tekst nog aanpassen naar nieuwe formulas

We gaan dat effe automatiseren want anders duurt dat veel te lang. 

In [1]:
import openpyxl
import pandas as pd
from openpyxl import load_workbook

## Monthly Return Index of Stocks

In [2]:
# Bestandsnaam van het Excel-bestand
file_name = 'DatesMonthlyReturnNeeded.xlsx'

# Het Excel-bestand lezen en opslaan als een DataFrame
df = pd.read_excel(file_name)

# De eerste paar rijen van de DataFrame printen om te bevestigen
df

,Index,Start,End,Freq
0,102,1999-01-01,2002-03-31,M
1,402,1999-04-01,2002-06-30,M
2,702,1999-07-01,2002-09-30,M
3,1002,1999-10-01,2002-12-31,M
4,103,2000-01-01,2003-03-31,M
...,...,...,...,...
84,123,2020-01-01,2023-03-31,M
85,423,2020-04-01,2023-06-30,M
86,723,2020-07-01,2023-09-30,M
87,1023,2020-10-01,2023-12-31,M


In [3]:
# De 'index' kolom converteren naar string, voorloopnullen toevoegen indien nodig
df['Index'] = df['Index'].astype(str).apply(lambda x: '0' + x if len(x) == 3 else x)

df

,Index,Start,End,Freq
0,0102,1999-01-01,2002-03-31,M
1,0402,1999-04-01,2002-06-30,M
2,0702,1999-07-01,2002-09-30,M
3,1002,1999-10-01,2002-12-31,M
4,0103,2000-01-01,2003-03-31,M
...,...,...,...,...
84,0123,2020-01-01,2023-03-31,M
85,0423,2020-04-01,2023-06-30,M
86,0723,2020-07-01,2023-09-30,M
87,1023,2020-10-01,2023-12-31,M


### For the stocks

In [4]:
# Bestandspad definiëren
file_path = 'Stocks_MonthlyReturn.xlsx'

# Een leeg Excel-bestand aanmaken met pandas ExcelWriter
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # We hoeven hier geen data toe te voegen, alleen het bestand initialiseren

    # Loop door elke rij in de DataFrame en maak een sheet voor elke 'Index'
    for index, row in df.iterrows():
        temp_df = pd.DataFrame({'A': ['']})  # Maak een tijdelijke lege DataFrame
        temp_df.to_excel(writer, sheet_name=row['Index'], index=False, header=False)

# Na het opslaan van het lege bestand, nu het werkboek bewerken om de formules toe te voegen
book = load_workbook(file_path)
for index, row in df.iterrows():
    
     # De start- en einddatum omzetten naar het gewenste formaat (zonder tijdstip)
    start_date_formatted = row['Start'].strftime('%Y-%m-%d')
    end_date_formatted = row['End'].strftime('%Y-%m-%d')
    
    # De formule opbouwen
    formula = f'=DSGRID("LDJSTOXX{row["Index"]}";"X(RI)~E";"{start_date_formatted}"; "{end_date_formatted}";"{row["Freq"]}";"RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;Curn=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;MonthlyTSFormat=False";"")'
    
    # De juiste sheet selecteren
    sheet = book[row['Index']]
    # Voeg de formule toe aan cel A1
    sheet['A1'] = "'"   +  formula #We voegen er dit aan toe anders tript die weer helemaal

# Sla de wijzigingen op
book.save(file_path)
book.close()